In [ ]:
! pip install langchain langchain-google-genai langchain_community tiktoken chromadb

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_ENDPOING']='https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY']='LANGCHAIN_API_KEY'
os.environ['GOOGLE_API_KEY']='GOOGLE_API_KEY'
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

# Multi Query
Our model will crate multiple number of query from user's query. This will help to LLM to fetch relevent answer from the vectorstore.
* Index

In [2]:
### Indexing ###
# Load blog
import bs4 # BeautifulSoup will help to provide you clean content from webpage
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs=loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300,chunk_overlap=50)

# Make split
splits=text_splitter.split_documents(blog_docs)

# Index
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore=Chroma.from_documents(documents=splits,embedding=GoogleGenerativeAIEmbeddings(model='models/embedding-001'))
retriever=vectorstore.as_retriever()

# Prompt

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI


# Multi Query: Different Perspectives -> We are creating multiple question from user's question for better simlarity search in vectorDB.
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}"""

prompt_perspectives=ChatPromptTemplate.from_template(template)

generate_queries=(
    prompt_perspectives
    | ChatGoogleGenerativeAI(model='gemini-1.5-flash',temperature=0)
    | StrOutputParser()
    | (lambda x:x.split('\n'))
)


In [4]:
from langchain.load import dumps,loads

def get_unique_union(documents:list[list]):
  """ Unique union of retrieved docs"""
  # Flatten list of lists, and convert each document to string.
  flattended_docs=[dumps(doc) for sublist in documents for doc in sublist]
  # Get unique documents
  unique_docs=list(set(flattended_docs))
  # Return
  return[loads(doc) for doc in unique_docs]

# Retrieve
question='What is task decomposistion for LLM agents ?'
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs=retrieval_chain.invoke({'question':question})
len(docs)

<ipython-input-4-06be9bc56585>:10: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return[loads(doc) for doc in unique_docs]


10

In [5]:
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='}\n]\nChallenges#\nAfter going through key ideas and demos of building LLM-centered agents, I start to see a couple common limitations:'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='GOALS:\n\n1. {{user-provided goal 1}}\n2. {{user-provided goal 2}}\n3. ...\n4. ...\n5. ...\n\nConstraints:\n1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.\n2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.\n3. No user assistance\n4. Exclusively use the commands listed in double quotes e.g. "command name"\n5. Use subprocesses for commands that will not terminate within a few minutes'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='

In [6]:
from operator import itemgetter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt=ChatPromptTemplate.from_template(template)
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash',temperature=0)

final_rag_chain=(
    {'context':retrieval_chain,
     'question':itemgetter('question')}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({'question':question})

'Task decomposition for LLM agents is the process of breaking down large, complex tasks into smaller, more manageable subgoals.  This allows the agent to handle complex problems more efficiently.  Methods include:\n\n* **Chain of Thought (CoT):**  A prompting technique instructing the model to "think step by step," decomposing the task into smaller steps.\n* **Tree of Thoughts:** An extension of CoT that explores multiple reasoning possibilities at each step, creating a tree structure of potential solutions.  Search can be breadth-first or depth-first.\n* **LLM prompting:**  Simple prompts like "Steps for XYZ. 1.", or "What are the subgoals for achieving XYZ?" can elicit task decomposition from the LLM.\n* **Task-specific instructions:**  Instructions tailored to the task type (e.g., "Write a story outline" for writing a novel).\n* **Human input:**  Humans can directly provide the task decomposition.'

# RAG Fusion-
Sends multiple versions of your question,  receives results for each question, mixes them and ranks them to get the best answer.

In [7]:
# Prompt
from langchain.prompts import ChatPromptTemplate

# RAG- Fusion
template="""You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion=ChatPromptTemplate.from_template(template)

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

# Creating chains -> It is a sequence of steps that take input,transform it into output
generate_queries=(
    prompt_rag_fusion
    | ChatGoogleGenerativeAI(model='gemini-1.5-flash')
    | StrOutputParser()
    | (lambda x:x.split('\n'))
)

In [9]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents
        and an optional parameter k used in the RRF formula """

    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

6

In [10]:
docs

[(Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first searc

In [11]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template="""Answer the following question based on this context:

{context}

Question: {question}
"""

prompt=ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM agents is the process of breaking down large, complex tasks into smaller, more manageable subgoals.  This can be achieved in several ways:  using simple prompts like "Steps for XYZ. 1.",  "What are the subgoals for achieving XYZ?"; using task-specific instructions (e.g., "Write a story outline" for writing a novel); or with human input.  Techniques like Chain of Thought (CoT) and Tree of Thoughts further enhance this process by guiding the LLM to think step-by-step or explore multiple reasoning possibilities at each step, respectively.'

# Decomposition-
We will break down user's query into set of sub-problems. LLM will generate multiple queries related to the question

In [13]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template="""
    You are a helpful assistant that generates multiple sub-question related to an input question. \n
    The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation \n
    Generate multiple search queries related to:{question} \n
    Output (3 queries):
"""
prompt_decomposition=ChatPromptTemplate.from_template(template)

In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# LLM
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

# Chain
generate_queries_decomposition=(prompt_decomposition | llm | StrOutputParser() | (lambda x:x.split('\n')))

# Run
question="What are the main components of an LLM-powered autonomous agent system ?"
questions=generate_queries_decomposition.invoke({'question':question})

In [18]:
questions

['Here are three search queries related to "What are the main components of an LLM-powered autonomous agent system?":',
 '',
 '1. **"Architecture of LLM-based autonomous agents"**: This query focuses on the overall structure and design principles.',
 '',
 '2. **"Key modules in autonomous agents using large language models"**: This query specifically targets the individual functional blocks within the system.',
 '',
 '3. **"LLM agent components: memory, planning, action execution"**: This query highlights three crucial aspects often found in such systems, prompting more specific results.']

# Answer Recursively

In [19]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question:

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [22]:
from operator import itemgetter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""

    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

# llm
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash',temperature=0)

q_a_pairs = ""
for q in questions:

    rag_chain = (
    {"context": itemgetter("question") | retriever,
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")}
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [23]:
answer

"Based on the provided text, an LLM agent comprises the following components related to memory, planning, and action execution:\n\n**1. Memory:**\n\n*   **Short-term memory:** This utilizes the LLM's in-context learning capabilities, essentially its immediate access to the current prompt and recent interactions.  It's limited (e.g., around 4000 words).\n*   **Long-term memory:**  This is handled by an external vector store and fast retrieval system, allowing the agent to retain and recall information indefinitely.  Strategies for determining memory importance (recency, relevance, importance as judged by the LLM itself) are employed. A retrieval model surfaces relevant context. A reflection mechanism synthesizes memories into higher-level inferences to guide future behavior.\n\n**2. Planning:**\n\n*   **Subgoal decomposition:** The agent breaks down complex tasks into smaller, manageable subgoals.\n*   **Reflection and refinement:** The agent self-critiques and reflects on past actions,

# Anser Individually

In [27]:
# Answer each sub-question individually

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""

    # Use our decomposition /
    sub_questions = sub_question_generator_chain.invoke({"question":question})

    # Initialize a list to hold RAG chain results
    rag_results = []

    for sub_question in sub_questions:

        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)

        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs,
                                                                "question": sub_question})
        rag_results.append(answer)

    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

<ipython-input-27-38821e4802e2>:23: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(sub_question)


In [28]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""

    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

"LLM-powered autonomous agent systems typically consist of several key components working in concert.  The core is the **Large Language Model (LLM)** itself, acting as the central controller.  This LLM is complemented by:\n\n* **Planning:** This module allows the agent to break down complex tasks into smaller, more manageable subgoals.  This often involves subgoal decomposition and iterative refinement, sometimes incorporating reflection to improve future planning.\n\n* **Memory:**  This is crucial for maintaining context and learning from past experiences.  It usually includes both short-term memory (often limited, as seen in examples with 4000-word limits) and long-term memory for storing information over extended periods.\n\n* **Tools:**  These are external resources the agent can interact with to achieve its goals.  This could include internet access, specific APIs, or other software tools.\n\n* **Natural Language Interface:** This facilitates communication between the agent and th

# Step Back

In [29]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [31]:
generate_queries_step_back = prompt | ChatGoogleGenerativeAI(model='gemini-1.5-flash',temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'How do LLMs handle complex tasks?'

In [32]:
# Response prompt
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatGoogleGenerativeAI(model='gemini-1.5-flash',temperature=0)
    | StrOutputParser()
)

chain.invoke({"question": question})

'Task decomposition, in the context of Large Language Model (LLM) agents, is the process of breaking down a complex task into smaller, more manageable sub-tasks.  This is crucial because LLMs, while powerful, have limitations in processing and reasoning about highly complex problems in a single step.  By decomposing the task, the agent can address each sub-task individually, making the overall problem tractable.\n\nSeveral methods exist for task decomposition:\n\n* **Chain of Thought (CoT):** This prompting technique instructs the LLM to "think step by step," explicitly outlining the reasoning process.  This forces the model to decompose the task implicitly through its step-by-step reasoning.\n\n* **Tree of Thoughts (ToT):**  An extension of CoT, ToT explores multiple reasoning paths at each step, creating a tree structure of potential solutions.  This allows the agent to consider various approaches and potentially find better solutions than a linear, single-path approach.  Search stra

# HyDE

In [33]:
from langchain.prompts import ChatPromptTemplate

# HyDE document genration
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

generate_docs_for_retrieval = (
    prompt_hyde | ChatGoogleGenerativeAI(model='gemini-1.5-flash',temperature=0) | StrOutputParser()
)

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question":question})

'Task decomposition, in the context of Large Language Model (LLM) agents, refers to the process of breaking down a complex, high-level task into a sequence of simpler, more manageable sub-tasks that are individually tractable for the LLM.  This decomposition is crucial because LLMs, while capable of impressive feats of language understanding and generation, often struggle with tasks requiring extensive reasoning, memory, or external interaction.  A complex task like "plan a surprise birthday party" cannot be directly addressed by an LLM; instead, it must be decomposed into sub-tasks such as "determine guest list," "select a venue," "create a budget," "design invitations," and "purchase supplies."  Each of these sub-tasks can then be further decomposed if necessary, creating a hierarchical task structure.  Effective task decomposition significantly improves the performance and reliability of LLM agents by mitigating the limitations of the underlying LLM and enabling the agent to leverag

In [34]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search

In [35]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retireved_docs,"question":question})

'Task decomposition, in the context of LLM-powered autonomous agents, is the process of breaking down large, complex tasks into smaller, more manageable sub-goals.  This allows the agent to handle complex problems more efficiently.  Methods for achieving this include:\n\n* **Chain of Thought (CoT):**  A prompting technique instructing the model to think step-by-step, decomposing the task into smaller steps.\n* **Tree of Thoughts (ToT):** An extension of CoT that explores multiple reasoning possibilities at each step, creating a tree structure of potential solutions.\n* **Simple prompting:**  Directly prompting the LLM with instructions like "Steps for XYZ." or "What are the subgoals for achieving XYZ?".\n* **Task-specific instructions:** Using instructions tailored to the task, such as "Write a story outline." for writing a novel.\n* **Human input:**  Directly providing the decomposition steps from a human.\n* **External classical planner (LLM+P):** Using an external planner (like one 